In [139]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import *
from catboost import datasets

In [140]:
train_df = pd.read_csv(r"train_s3TEQDk.csv")
test_df  = pd.read_csv(r"test_mSzZ8RL.csv")

In [141]:
train_df=train_df.drop('ID',1)
test_df=test_df.drop('ID',1)

In [142]:
train_df.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [143]:
train_df.isnull().sum()

Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         29325
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [144]:
train_df['Credit_Product'] = train_df['Credit_Product'].fillna('new')
test_df['Credit_Product'] = test_df['Credit_Product'].fillna('new')

In [145]:
#Label Encoding
# above i have checked the dtype and i am now converting all the object and bool to string for further use.
from sklearn.preprocessing import LabelEncoder 

train_df['Gender']=train_df['Gender'].astype(str)
train_df['Region_Code']=train_df['Region_Code'].astype(str)
train_df['Occupation']=train_df['Occupation'].astype(str)
train_df['Channel_Code']=train_df['Channel_Code'].astype(str)
train_df['Credit_Product']=train_df['Credit_Product'].astype(str)
train_df['Is_Active']=train_df['Is_Active'].astype(str)


# converting all the categories to numeric format as model does not take data in english.

train_df['Gender']=LabelEncoder().fit_transform(train_df['Gender'])
train_df['Region_Code']=LabelEncoder().fit_transform(train_df['Region_Code'])
train_df['Occupation']=LabelEncoder().fit_transform(train_df['Occupation'])
train_df['Channel_Code']=LabelEncoder().fit_transform(train_df['Channel_Code'])
train_df['Credit_Product']=LabelEncoder().fit_transform(train_df['Credit_Product'])
train_df['Is_Active']=LabelEncoder().fit_transform(train_df['Is_Active'])



In [146]:
#Label Encoding
# above i have checked the dtype and i am now converting all the object and bool to string for further use.
from sklearn.preprocessing import LabelEncoder 

test_df['Gender']=train_df['Gender'].astype(str)
test_df['Region_Code']=train_df['Region_Code'].astype(str)
test_df['Occupation']=train_df['Occupation'].astype(str)
test_df['Channel_Code']=train_df['Channel_Code'].astype(str)
test_df['Credit_Product']=train_df['Credit_Product'].astype(str)
test_df['Is_Active']=train_df['Is_Active'].astype(str)


# converting all the categories to numeric format as model does not take data in english.

test_df['Gender']=LabelEncoder().fit_transform(test_df['Gender'])
test_df['Region_Code']=LabelEncoder().fit_transform(test_df['Region_Code'])
test_df['Occupation']=LabelEncoder().fit_transform(test_df['Occupation'])
test_df['Channel_Code']=LabelEncoder().fit_transform(test_df['Channel_Code'])
test_df['Credit_Product']=LabelEncoder().fit_transform(test_df['Credit_Product'])
test_df['Is_Active']=LabelEncoder().fit_transform(test_df['Is_Active'])



In [147]:
train_df.tail()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
245720,1,51,34,3,2,109,2,1925586,0,0
245721,1,27,18,2,0,15,0,862952,1,0
245722,0,26,31,2,0,13,0,670659,0,0
245723,0,28,23,2,0,31,0,407504,0,0
245724,1,29,19,2,0,21,0,1129276,0,0


In [148]:
train_df.isnull().sum()

Gender                 0
Age                    0
Region_Code            0
Occupation             0
Channel_Code           0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
Is_Lead                0
dtype: int64

In [149]:
y = train_df.Is_Lead
X = train_df.drop('Is_Lead', axis=1)

In [150]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [151]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [152]:
# Data Scaling/normalization of the features that will go to the NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test_df = sc.transform(test_df)

In [153]:
# import XGBClassifier
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)



# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

C:\Users\Admin\anaconda3\envs\tf_gpu1\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:58:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1.0, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [156]:
from sklearn.metrics import roc_auc_score
ypred = xgb_clf.predict(X)
roc_auc_score(ypred,y)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'] ['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active']
expected f2, f0, f3, f4, f1, f8, f6, f5, f7 in input data
training data did not have the following fields: Credit_Product, Gender, Is_Active, Avg_Account_Balance, Region_Code, Age, Occupation, Vintage, Channel_Code

In [157]:
sat1 = pd.read_csv(r"sample_submission_eyYijxG.csv")
sat1['Is_Lead'] = xgb_clf.predict(test_df)
sat1.to_csv('submisssion1.csv',index=False)

In [ ]:
# build the lightgbm model
#!pip install lightgbm
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score
ypred = clf.predict(X)
roc_auc_score(ypred,y)

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [158]:
#----------------------- Building the model -----------------------#

# Initializing the ANN by calling the Sequential class fromm keras of Tensorflow
ann = tf.keras.models.Sequential()


#----------------------------------------------------------------------------------
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 500, activation = 'relu'))


#----------------------------------------------------------------------------------
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 300, activation = 'relu'))

ann.add(tf.keras.layers.Dropout(0.3))



#----------------------------------------------------------------------------------
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [161]:
#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN model on training set  (fit method always the same) 
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 1032, epochs = 1000)

Epoch 1/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.3209 - accuracy: 0.8701
Epoch 2/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.8707
Epoch 3/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3198 - accuracy: 0.8703
Epoch 4/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.3195 - accuracy: 0.8708
Epoch 5/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3195 - accuracy: 0.8708
Epoch 6/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3196 - accuracy: 0.8710
Epoch 7/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.3196 - accuracy: 0.8709
Epoch 8/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3194 - accuracy: 0.8705
Epoch 9/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3191 - accuracy: 0.8711
Epoch 10/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.319

191/191 [==============================] - 0s 3ms/step - loss: 0.3103 - accuracy: 0.8748
Epoch 80/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3096 - accuracy: 0.8745
Epoch 81/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3100 - accuracy: 0.8747
Epoch 82/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3097 - accuracy: 0.8744
Epoch 83/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3100 - accuracy: 0.8747
Epoch 84/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3090 - accuracy: 0.8747
Epoch 85/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3094 - accuracy: 0.8745
Epoch 86/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3091 - accuracy: 0.8743
Epoch 87/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3092 - accuracy: 0.8747
Epoch 88/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3093 - a

191/191 [==============================] - 0s 3ms/step - loss: 0.3004 - accuracy: 0.8780
Epoch 159/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.3004 - accuracy: 0.8781
Epoch 160/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.2999 - accuracy: 0.8787
Epoch 161/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3000 - accuracy: 0.8779
Epoch 162/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.2997 - accuracy: 0.8780
Epoch 163/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3000 - accuracy: 0.8782
Epoch 164/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.3007 - accuracy: 0.8778
Epoch 165/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.2995 - accuracy: 0.8785
Epoch 166/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.2994 - accuracy: 0.8786
Epoch 167/1000
191/191 [==============================] - 0s 2ms/step - loss: 0

191/191 [==============================] - 0s 3ms/step - loss: 0.2911 - accuracy: 0.8816
Epoch 237/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2914 - accuracy: 0.8815
Epoch 238/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2903 - accuracy: 0.8827
Epoch 239/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2909 - accuracy: 0.8817
Epoch 240/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2906 - accuracy: 0.8820
Epoch 241/1000
191/191 [==============================] - 0s 2ms/step - loss: 0.2907 - accuracy: 0.8822
Epoch 242/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2906 - accuracy: 0.8825
Epoch 243/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2911 - accuracy: 0.8820
Epoch 244/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2900 - accuracy: 0.8826
Epoch 245/1000
191/191 [==============================] - 0s 3ms/step - loss: 0

191/191 [==============================] - 1s 3ms/step - loss: 0.2829 - accuracy: 0.8850
Epoch 315/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2829 - accuracy: 0.8856
Epoch 316/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2830 - accuracy: 0.8854
Epoch 317/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2819 - accuracy: 0.8847
Epoch 318/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2825 - accuracy: 0.8856
Epoch 319/1000
191/191 [==============================] - 1s 4ms/step - loss: 0.2830 - accuracy: 0.8847
Epoch 320/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2823 - accuracy: 0.8853
Epoch 321/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2822 - accuracy: 0.8850
Epoch 322/1000
191/191 [==============================] - 0s 3ms/step - loss: 0.2820 - accuracy: 0.8863
Epoch 323/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

...

191/191 [==============================] - 1s 3ms/step - loss: 0.2563 - accuracy: 0.8956
Epoch 627/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2566 - accuracy: 0.8959
Epoch 628/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2560 - accuracy: 0.8958
Epoch 629/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2562 - accuracy: 0.8961
Epoch 630/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2564 - accuracy: 0.8954
Epoch 631/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2559 - accuracy: 0.8963
Epoch 632/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2563 - accuracy: 0.8958
Epoch 633/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2559 - accuracy: 0.8957
Epoch 634/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2552 - accuracy: 0.8960
Epoch 635/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

191/191 [==============================] - 1s 3ms/step - loss: 0.2514 - accuracy: 0.8974
Epoch 705/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2501 - accuracy: 0.8977
Epoch 706/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2501 - accuracy: 0.8979
Epoch 707/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2500 - accuracy: 0.8982
Epoch 708/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2508 - accuracy: 0.8979
Epoch 709/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2504 - accuracy: 0.8980
Epoch 710/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2503 - accuracy: 0.8979
Epoch 711/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2508 - accuracy: 0.8977
Epoch 712/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2510 - accuracy: 0.8971
Epoch 713/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

191/191 [==============================] - 1s 3ms/step - loss: 0.2460 - accuracy: 0.8998
Epoch 783/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2469 - accuracy: 0.8993
Epoch 784/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2459 - accuracy: 0.8996
Epoch 785/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2457 - accuracy: 0.8997
Epoch 786/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2457 - accuracy: 0.8999
Epoch 787/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2446 - accuracy: 0.9001
Epoch 788/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2466 - accuracy: 0.8998
Epoch 789/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2459 - accuracy: 0.9000
Epoch 790/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2463 - accuracy: 0.9003
Epoch 791/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

191/191 [==============================] - 1s 3ms/step - loss: 0.2415 - accuracy: 0.9014
Epoch 861/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2424 - accuracy: 0.9004
Epoch 862/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2406 - accuracy: 0.9015
Epoch 863/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2410 - accuracy: 0.9016
Epoch 864/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2409 - accuracy: 0.9019
Epoch 865/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2410 - accuracy: 0.9018
Epoch 866/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2409 - accuracy: 0.9014
Epoch 867/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2417 - accuracy: 0.9017
Epoch 868/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2412 - accuracy: 0.9014
Epoch 869/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

191/191 [==============================] - 1s 3ms/step - loss: 0.2359 - accuracy: 0.9040
Epoch 939/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2368 - accuracy: 0.9026
Epoch 940/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2369 - accuracy: 0.9032
Epoch 941/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2363 - accuracy: 0.9036
Epoch 942/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2373 - accuracy: 0.9034
Epoch 943/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2368 - accuracy: 0.9032
Epoch 944/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2365 - accuracy: 0.9037
Epoch 945/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2356 - accuracy: 0.9040
Epoch 946/1000
191/191 [==============================] - 1s 3ms/step - loss: 0.2363 - accuracy: 0.9025
Epoch 947/1000
191/191 [==============================] - 1s 3ms/step - loss: 0

In [162]:
sat1 = pd.read_csv(r"sample_submission_eyYijxG.csv")
sat1['Is_Lead'] = ann.predict(X_train)
sat1.to_csv('submisssion1.csv',index=False)

In [188]:
from sklearn.metrics import roc_auc_score
y_pred_prob= ann.predict(test_df)
y_pred = (y_pred_prob > 0.4)

sat1 = pd.read_csv(r"sample_submission_eyYijxG.csv")
sat1['Is_Lead'] = y_pred
sat1.to_csv('submisssion1.csv',index=False)


In [179]:
#Confusion Matrix
confusion_matrix1 = confusion_matrix(y_train, y_pred)
print("Confusion Matrix", confusion_matrix1)
print("Accuracy Score", accuracy_score(y_train, y_pred))

TypeError: 'numpy.ndarray' object is not callable

In [184]:
roc_auc_score(y_train, y_pred)

0.8775134612765262